In [17]:
from nrl8 import *

data = np.loadtxt("classification_train.csv", delimiter=',', skiprows=1)
X_train = data[:, 2:]
y_train = data[:, 1]
m, n = X_train.shape
ctgrs = np.unique(y_train) # np.unique() to get categories
q = len(ctgrs)

# mean = np.mean(X_train, axis=0).reshape(1, n)
# ptp = np.max(X_train, axis=0) - np.min(X_train, axis = 0)
# X_train = (X_train - mean)/ptp

data = np.loadtxt("classification_train.csv", delimiter=',', skiprows=1)
X_train = data[:, 2:]
y_train = data[:, 1]
# X_train, y_train = spiral_data(samples=100, classes=3)
m, n = X_train.shape
ctgrs = np.unique(y_train) # np.unique() to get categories
q = len(ctgrs)

# mean = np.mean(X_train, axis=0).reshape(1, n)
# ptp = np.max(X_train, axis=0) - np.min(X_train, axis = 0)

# X_train = (X_train - mean)/ptp

while True:
    l = input("Number of layers: ")
    if l.isdigit() and int(l) > 0:
        break
    else:
        print(f"Positive integer expected...")
        continue
n_l = int(l)


iters = iterations()
learn_rate = alpha()

activation_layer = Activation_ReLU()
activation_last_layer = Activation_Softmax()
activation_loss = Actvn_Softmax_Loss_CategoricalCrossentropy()
optimiser = SGD(learn_rate)

n_inputs = int(X_train.shape[1])


layer_lst = []

for i in range(n_l):
    n_neurons = neurons()
    if i == 0:
        layer = Gen_layer(n_inputs, n_neurons)
        layer_lst.append(layer)
        # layer.forward(X_train)

        # out = actvn.forward(layer.output)
        # activations.append(out)
    if 0 < i <= n_l-1:
        layer1 =Gen_layer(layer_lst[-1].n_neurons, n_neurons)
        layer_lst.append(layer1)

        # actvn1.forward(activations[i-1].output)
        # activations.append(actvn1.forward(layer1.output))
    # elif i == n_l-1:
    #     activation = Activation_Softmax()


l1 = len(layer_lst)
print(f"Number of layers: {l1} | alpha: {learn_rate}")

for i in range(l1): 
    print(f"Layer {i + 1} : {layer_lst[i].n_neurons} neurons")

for iter in range(iters):
    activations = []
    for i in range(n_l):
        if 0<= i < n_l-1:
            activations.append(Activation_ReLU())
        if i == n_l -1:
            activations.append(Activation_Softmax())
    l = len(layer_lst)
    for j in range(l):
        if j == 0:
            layer_lst[j].forward(X_train)
            activations[j].forward(layer_lst[j].output)
            # activations.append(actvn_j)
        if 0 < j < l -1:
            layer_lst[j].forward(activations[j-1].output)
            activations[j].forward(layer_lst[j].output)
            # activations.append(actvn_j)

        if j == l - 1:
            layer_lst[j].forward(activations[j-1].output)
            activations[j].forward(layer_lst[j].output)
            # activations.append(actvn_j)
        
    # Loss
    loss = activation_loss.forward(layer_lst[l-1].output, y_train)

    # Accuracy
    predictions = np.argmax(activation_loss.output, axis = 1)
    if len(y_train.shape) == 2:
        y_train = np.argmax(y_train, axis = 1)
    accuracy = np.mean(predictions==y_train)

    if iter % 100 == 0:
        print(f"Iteration no. {iter}: " + f"loss {loss:0.4e} " + f"acc: {accuracy:.3f}")
    
    # Last iterations to check variation
    if (iters - 10) <= iter <= (iters - 1):
        print(f"Iteration no. {iter}: " + f"loss {loss:0.4e} " + f"acc: {accuracy:.3f}")  

    lst_der_inputs = []
    activation_loss.backward(activation_loss.output, y_train)

    for k in range(l):
        if k == 0:
            layer_lst[l-1].backward(activation_loss.dinputs)
            der = activations[l-2].backward(layer_lst[l-1].dinputs)
            lst_der_inputs.append(der)
        if 0 < k < l - 1:
                layer_lst[l-1-k].backward(activations[l-1-k].dinputs)
                der = activations[l-2-k].backward(layer_lst[l-1-k].dinputs)
                lst_der_inputs.append(der)
        if (k == l-1):
            layer_lst[0].backward(activations[0].dinputs)
    for a in range(l):
        optimiser.update_p(layer_lst[a])

Number of layers: 2 | alpha: 0.001
Layer 1 : 784 neurons
Layer 2 : 10 neurons
Iteration no. 0: loss 1.5030e+01 acc: 0.066


In [11]:
activations

In [10]:
l

4

In [2]:
import pandas as pd
test_data = np.loadtxt("classification_test.csv", delimiter=',', skiprows=1)
X_test = test_data[:, 1:]

l = len(layer_lst)

for j in range(l):
    if j == 0:
        layer_lst[j].forward(X_test)
        activation_layer.forward(layer_lst[j].output)

    if 0 < j < l -1:
        layer_lst[j].forward(activation_layer.output)
        activation_layer.forward(layer_lst[j].output)

    if j == l - 1:
        layer_lst[j].forward(activation_layer.output)
        activation_last_layer.forward(layer_lst[j].output)

df = pd.DataFrame(test_data[:, 0])
df['Output'] = np.argmax(activation_last_layer.output, axis=1)
df.columns = ['ids', 'Output']

print(df)

# print(f"Class: {np.argmax(activation_last_layer.output)}")

NameError: name 'layer_lst' is not defined

In [21]:
df.to_csv("submission_neural_2layer.csv")

In [19]:
print(f"Class_test: {np.argmax(activation_last_layer.output, axis = 1)}")

Class_test: [6 1 4 ... 8 2 6]


In [6]:
a = np.array([[1, 2, 3],
              [3, 5, 6],
              [4, 3, 7]])
np.argmax(a, axis = 0)

array([2, 1, 2], dtype=int64)